In [9]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression

In [7]:
des_concat_train = np.load('./features/descriptions_train_concat_embed_vectors.npy')
des_concat_test = np.load('./features/descriptions_test_concat_embed_vectors.npy')
des_long_train = np.load('./features/descriptions_train_long_embed_vectors.npy')
des_long_test = np.load('./features/descriptions_test_long_embed_vectors.npy')
image_1000_train = np.load('./features/image_features_1000_train.npy')
image_1000_test = np.load('./features/image_features_1000_test.npy')
image_2048_train = np.load('./features/image_features_2048_train.npy')
image_2048_test = np.load('./features/image_features_2048_test.npy')

IOError: Failed to interpret file './features/descriptions_train_concat_embed_vectors.npy' as a pickle

In [8]:
def get_nearest_20(vec, vec_set):
    dis = [np.linalg.norm(np.array(vec) - np.array(x)) for x in vec_set]
    return np.argsort(dis)[:20]

def map20score(y, pred):
    if y in pred:
        return (20 - pred.tolist().index(y)) / 20.0
    else:
        return 0

def map20eval(pls, des_vectors, image_vectors):
    image_pred = pls.predict(des_vectors)
    top_20 = [get_nearest_20(vec, image_vectors) for vec in image_pred]
    print(len(top_20))
    scores = [map20score(i, top_20[i]) for i in range(len(top_20))]
    print "score: %f" % np.mean(scores)
    return scores

In [4]:
pls_concat_1000 = PLSRegression(n_components=256)
pls_concat_1000.fit(des_concat_train, image_1000_train)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [6]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]

In [12]:
def output_submission(top_20, output_path):
    top_20_image_IDs = map(lambda x: ' '.join(map(lambda xx: str(xx) + '.jpg', x)), top_20)
    description_ID = map(lambda x: str(x) + '.txt', range(len(top_20)))
    submission_df = pd.DataFrame({'Descritpion_ID': description_ID, 'Top_20_Image_IDs': top_20_image_IDs})
    submission_df.to_csv(output_path, index=False)


In [13]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000.csv')

In [14]:
pls_concat_1000 = PLSRegression(n_components=256, max_iter=1000)
pls_concat_1000.fit(des_concat_train, image_1000_train)

PLSRegression(copy=True, max_iter=1000, n_components=256, scale=True,
       tol=1e-06)

In [15]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]


In [16]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000_1000iter.csv')

In [40]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun.npy')

In [56]:
pls_BOW_noun_1000 = PLSRegression(n_components=10)
pls_BOW_noun_1000.fit(des_BOW_noun_train[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train[8000:], image_1000_train[8000:])

0.3791097044896606


In [49]:
scores = map20eval(pls_BOW_noun_1000, des_BOW_noun_train[8000:], image_1000_train[8000:])

2000
score: 0.290600


In [51]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [52]:
pred_BOW_noun_1000 = pls_BOW_noun_1000.predict(des_BOW_noun_test)
top_20_BOW_noun_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000]

In [53]:
output_submission(top_20_BOW_noun_1000, 'submissions/PLSR_256_BOW_noun_1000.csv')

In [59]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun_5452.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun_5452.npy')

In [60]:
from sklearn.decomposition import PCA

In [89]:
pca = PCA(n_components=512)

In [90]:
pca.fit(des_BOW_noun_train)

PCA(copy=True, iterated_power='auto', n_components=512, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [91]:
des_BOW_noun_train_pca = pca.transform(des_BOW_noun_train)
des_BOW_noun_test_pca = pca.transform(des_BOW_noun_test)

In [92]:
print des_BOW_noun_train_pca.shape

(10000, 512)


In [93]:
pls_BOW_noun_1000 = PLSRegression(n_components=20)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train_pca[8000:], image_1000_train[8000:])

0.43198989225672163


In [96]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [97]:
pred_BOW_noun_1000_5452 = pls_BOW_noun_1000.predict(des_BOW_noun_test_pca)
top_20_BOW_noun_1000_5452 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000_5452]

In [99]:
output_submission(top_20_BOW_noun_1000_5452452, 'submissions/PLSR_256_BOW_noun_1000_5452_pca_512.csv')

In [80]:
"""
Luffy part.
"""

import pickle
import os.path
from sklearn.decomposition import PCA

train_description_feature_map = pickle.load(open('./features/11_30_[train]_description_normalized_only_>1_vector_py27.pkl', 'rb'))
test_derscription_feature_map = pickle.load(open('./features/11_30_[test]_description_normalized_only_>1_vector_py27.pkl', 'rb'))

train_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], train_description_feature_map))
test_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], test_derscription_feature_map))

pca_model_name = "./models/pca_bow_all_to_2048.pkl"
if os.path.exists(pca_model_name):
    pca = pickle.load(open(pca_model_name, 'rb'))
else:
    # train PCA.
    pca = PCA(n_components=3500)
    pca.fit(train_bow_lst)
    pickle.dump(pca, open(pca_model_name, 'wb'), protocol=2)

des_BOW_all_train_pca = pca.transform(train_bow_lst)
des_BOW_all_test_pca = pca.transform(test_bow_lst)

print des_BOW_all_test_pca.shape

(2000, 2048)


In [81]:
train_img_map = pickle.load(open('./features/12_1_train_img_feature_map_py27.pkl', 'rb'))
test_img_map = pickle.load(open('./features/12_1_test_img_feature_map_py27.pkl', 'rb'))
print "loaded img feature map..."

In [82]:
train_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], train_img_map)))
test_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], test_img_map)))

In [95]:
model_name = "./models/pls_1000_pool5_to_BOW_all_pca_5816.pkl"
if os.path.exists(model_name):
    model = pickle.load(open(model_name, 'rb'))
else:
    # train PLSR.
    model = PLSRegression(n_components=800)
    model.fit(train_pool5_img[:8000], des_BOW_all_train_pca[:8000])
    pickle.dump(model, open(model_name, 'wb'), protocol=2)
    print model.score(train_pool5_img[8000:], des_BOW_all_train_pca[8000:])
print "model loaded..."

model loaded...


In [96]:
from scipy.spatial.distance import cdist
import csv

pred_test_pool5 = model.predict(test_pool5_img)

dist = cdist(test_bow_lst, pred_test_pool5, 'cosine') # CHANGE!!
print("description * images dist matrix, shape:", dist.shape)
sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.

with open('./submission/submission_plsr_1000_pool5_to_bow_all_pca_5816.csv', 'w') as csvfile:
        # write csv header
        fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for i, row in enumerate(sorted_id):
            top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
            res = {}
            res['Descritpion_ID'] = str(i) + ".txt" # file name
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

print("Writing Complete.")

('description * images dist matrix, shape:', (2000, 2000))
Writing Complete.


In [34]:
"""
Text preprocessing.
"""
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

nltk.download('stopwords')

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words


[nltk_data] Downloading package stopwords to /Users/EL-
[nltk_data]     CAPITAN-2016/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
"""
Use tags overlap to re-rank.

- test_derscription_feature_map ['description_n_cleaned']
- count image tags total apperances / ['description_n_cleaned'] length in above set. 
- add to dist matrix, then sorting.
"""
# print test_derscription_feature_map[0]
for t in test_derscription_feature_map:
    words = t["description_n"].split(" ")
    words = normalize(words)
    t["description_n_cleaned"] = words

print test_derscription_feature_map[0]

{u'description_complete': u'A woman walking down a street past a doorway. a woman is walking past a doorway on a sidewalk A woman talking on a cell phone checks her watch. A woman talks on a cell phone and checks her watch while walking on a sidewalk. A woman wearing blue is on the phone and walking along a sidewalk in front of a building with black planters on both sides of the entrance.', u'name': u'0.txt', u'description_n_v_adj': u'woman street doorway. woman doorway sidewalk woman cell phone watch. woman cell phone watch sidewalk. woman blue phone walking sidewalk front building black entrance.', 'description_n_cleaned': [u'woman', u'street', u'doorway', u'woman', u'doorway', u'sidewalk', u'woman', u'cell', u'phone', u'watch', u'woman', u'cell', u'phone', u'watch', u'sidewalk', u'woman', u'blue', u'phone', u'walking', u'sidewalk', u'front', u'building', u'entrance'], u'BOW_all_normalized_vector': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [77]:
print dist[0]
print np.argsort(dist)[0]
test_img_tags = pickle.load(open("./features/12_1_test_img_tags.pkl", "rb"))

new_dist = np.zeros(dist.shape)

def description_map_to_tags(words):
    """
    A set of rules to map certain descriptions nouns into tags wordding.
    such as:
    "man", "guy", "woman" => "person"
    """
    new_words = set()
    for w in words:
        if w == "man" or w == "woman" or w == "guy":
            new_words.add("person")
        else:
            new_words.add(w)
    return list(new_words)

for d_id, description_row in enumerate(dist):
    description_n_cleaned = test_derscription_feature_map[d_id]["description_n_cleaned"]
    description_n_cleaned = description_map_to_tags(description_n_cleaned) # hand-crafted map book.
    test_derscription_feature_map[d_id]["description_n_cleaned"] = description_n_cleaned
    sorted_ids = np.argsort(description_row)
    for i_id, img_id in enumerate(sorted_ids):
        img_tags = test_img_tags[img_id]["tags"]
        this_img_apperance_in_description = 0
        for tag in img_tags:
            this_img_apperance_in_description += description_n_cleaned.count(tag)
        this_img_score = float(this_img_apperance_in_description) / len(description_n_cleaned)
        new_dist[d_id][i_id] = dist[d_id][i_id] - 0.5 * this_img_score # re-rank dist matrix by adding tag overlap information.
print("after re-ranking...")
print new_dist[0]
print np.argsort(new_dist)[0]

[1.06474239 1.10100044 1.04627467 ... 1.05419919 1.12357977 1.08135871]
[1105  105 1480 ...  631 1474 1177]
after re-ranking...
[0.93974239 1.05933377 0.92127467 ... 1.05419919 1.08191311 1.08135871]
[ 105 1480 1105 ... 1779 1177 1474]


In [90]:
test_img_captions = pickle.load(open("./features/12_1_test_image_caption.pkl", "rb"))

In [94]:
print test_img_tags[833]["tags"]
print test_img_captions[833]
print "\n"
print test_derscription_feature_map[1]["description_n_cleaned"]

['cake', 'dining', 'table']
{u'caption': u'a piece of cake with a fork and a fork .', u'name': u'833.jpg'}


[u'slice', u'food', u'cake', u'top', u'plate', u'plate', u'slice', u'cake', u'quarter', u'cake', u'plate', u'piece', u'cake', u'plate', u'slab', u'sponge', u'cake', u'flowery', u'plate']


In [76]:
new_sorted_id = np.argsort(new_dist)
with open('./submission/submission_plsr_600_pool5_to_bow_all_pca_1024+rerank_by_tag_overlap.csv', 'w') as csvfile:
    # write csv header
    fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i, row in enumerate(new_sorted_id):
        top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
        res = {}
        res['Descritpion_ID'] = str(i) + ".txt" # file name
        res['Top_20_Image_IDs'] = " ".join(top_choices)
        writer.writerow(res)

print("Writing Complete.")

Writing Complete.


In [48]:
# import os
# test_img_tags = []
# test_image_tag_base = "./../../all/data/tags_test/"
# for file in os.listdir(test_image_tag_base):
#     if file.endswith(".txt"):
#         obj = {}
#         obj["name"] = file
#         index = file.split(".")[0]
#         obj["id"] = int(index)
#         file_name = test_image_tag_base + file
#         with open(file_name) as f:
#             content = f.readlines()
#             content = [x.strip().split(":")[1].split(" ") for x in content]
#             content = [item for sublist in content for item in sublist]
#         obj["tags"] = content
#         test_img_tags.append(obj)
# test_img_tags = sorted(test_img_tags, key=lambda x:x["id"])
# pickle.dump(test_img_tags, open("./features/12_1_test_img_tags.pkl", 'wb'), protocol=2)
# print test_img_tags[0]

{'tags': ['bed', 'backpack', 'suitcase', 'tie'], 'name': '0.txt', 'id': 0}


In [89]:

# test_img_captions = sorted(test_img_captions, key=lambda x: int(x["name"].split(".")[0]))
# pickle.dump(test_img_captions, open("./features/12_1_test_image_caption.pkl", 'wb'), protocol=2)